## Impots

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime

## Path

In [0]:
%run ./00_setup

In [0]:
raw_df = spark.read.parquet(raw_input_path).select("*", "_metadata.file_path")
display(raw_df.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,file_path
1,2022-12-01T00:37:35.000,2022-12-01T00:47:35.000,1.0,2.0,1.0,N,170,237,1,8.5,3.0,0.5,3.1,0.0,0.3,15.4,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet
1,2022-12-01T00:34:35.000,2022-12-01T00:55:21.000,0.0,8.4,1.0,N,138,141,2,26.0,4.25,0.5,0.0,0.0,0.3,31.05,2.5,1.25,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet
1,2022-12-01T00:33:26.000,2022-12-01T00:37:34.000,1.0,0.8,1.0,N,140,140,1,5.0,3.0,0.5,1.76,0.0,0.3,10.56,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet
1,2022-12-01T00:45:51.000,2022-12-01T00:53:16.000,1.0,3.0,1.0,N,141,79,3,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet
2,2022-12-01T00:49:49.000,2022-12-01T00:54:13.000,1.0,0.76,1.0,N,261,231,1,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet


## paths

## add metadata

In [0]:
ingest_time = current_timestamp()

raw_with_meta = raw_df.withColumn("ingest_time",ingest_time)    \
        .withColumn('ingest_date',to_date(ingest_time)) \
        .withColumnRenamed('file_path','_file_path')

display(raw_with_meta.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,_file_path,ingest_time,ingest_date
1,2022-12-01T00:37:35.000,2022-12-01T00:47:35.000,1.0,2.0,1.0,N,170,237,1,8.5,3.0,0.5,3.1,0.0,0.3,15.4,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:21.918Z,2025-12-12
1,2022-12-01T00:34:35.000,2022-12-01T00:55:21.000,0.0,8.4,1.0,N,138,141,2,26.0,4.25,0.5,0.0,0.0,0.3,31.05,2.5,1.25,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:21.918Z,2025-12-12
1,2022-12-01T00:33:26.000,2022-12-01T00:37:34.000,1.0,0.8,1.0,N,140,140,1,5.0,3.0,0.5,1.76,0.0,0.3,10.56,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:21.918Z,2025-12-12
1,2022-12-01T00:45:51.000,2022-12-01T00:53:16.000,1.0,3.0,1.0,N,141,79,3,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:21.918Z,2025-12-12
2,2022-12-01T00:49:49.000,2022-12-01T00:54:13.000,1.0,0.76,1.0,N,261,231,1,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5,0.0,dbfs:/Volumes/workspace/sathish/trip/input/yellow_tripdata_2022-12.parquet,2025-12-12T09:47:21.918Z,2025-12-12


## write bronze

In [0]:
bronze_write = raw_with_meta.write\
        .format("delta")    \
        .mode("overwrite")  \
        .partitionBy('ingest_date') \
        .save(bronze_write_path)

